In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

from tqdm import tqdm


In [11]:
def get_url(url):
#     try:
    # print(url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/78.0.3904.108 Chrome/78.0.3904.108 Safari/' + str(random.randrange(500,800)) + '.36'
    }
    r = requests.get(url, headers=headers)
    # print(r)
    return r.text
    
#     except Exception as e:
#         print(e)
#         return None
#     else:
#         return r.text
    
def get_base_url(term):
    return 'https://www.sejm.gov.pl/Sejm8.nsf/'

def get_single_vote_data(url, base_url):
    bs = BeautifulSoup(get_url(url), 'lxml')

    tab = bs.table
    tbody = tab.tbody
    parties = []
    voting_data = []

    for row in tbody.find_all('tr'):
        this_a = row.find_all('td')[0].a
        parties.append([this_a.text, this_a.get('href')])

    for party in parties:
        url_to_get = party[1]
        
        time.sleep(1.5)
        
        this_voting = BeautifulSoup(get_url(base_url + url_to_get), 'lxml')
        voting_rows = this_voting.table.tbody.find_all('tr')
        for voting_row in voting_rows:
            results = voting_row.find_all('td')
            voting_data.append([ party[0], results[1].text, results[2].text ])
            if len(results) > 3:
                voting_data.append([ party[0], results[4].text, results[5].text ])

    return pd.DataFrame(voting_data, columns=['party','person','vote'])

In [3]:
#all_votes = pd.read_hdf('all_votes.h5')
all_votes = pd.read_hdf('all_votes_pending.h5')
all_votes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14530 entries, 0 to 14529
Data columns (total 7 columns):
nr_glos           14530 non-null int64
url               14530 non-null object
opis              14530 non-null object
nr_posiedzenia    14530 non-null int64
kadencja          14530 non-null int64
timestamp         14530 non-null datetime64[ns]
saved_as          14530 non-null object
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 908.1+ KB


In [4]:
all_votes.sample(5)

,nr_glos,url,opis,nr_posiedzenia,kadencja,timestamp,saved_as
8465,100,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,Pkt 34. porz. dzien. Sprawozdanie Komisji w sp...,67,8,2018-07-20 12:55:50,
12161,63,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,Pkt 15. porz. dzien. Sprawozdanie Komisji o rz...,31,8,2016-11-30 11:27:22,
10209,82,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,Pkt 3. porz. dzien. Sprawozdanie Komisji o pos...,54,8,2017-12-14 22:38:18,
7271,50,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,Pkt 19. porz. dzien. Sprawozdanie Komisji o uc...,77,8,2019-02-20 23:08:38,
2062,14,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,Pkt 5. porz. dzien. Sprawozdanie Komisji o rzą...,81,7,2014-12-05 10:41:29,7_81_14.csv


In [12]:
#all_votes['saved_as'] = ''
# all_votes.to_hdf('all_votes.h5', key='all_votes', mode='w')
# all_votes.shape
all_votes[ all_votes['saved_as'] == '' ].shape

(3655, 7)

In [13]:
for index, row in tqdm(all_votes[ all_votes['saved_as'] == '' ].iterrows()):
    # print(row['url'])
    try:
        this_vote = get_single_vote_data(row['url'], get_base_url(row['kadencja'])) 

        this_filename = str(row['kadencja']) + '_' + str(row['nr_posiedzenia']) + '_' + str(row['nr_glos']) + '.csv'
        this_vote.to_csv('./votes/' + this_filename)

        all_votes.at[index, 'saved_as'] = this_filename
        all_votes.to_hdf('all_votes_pending.h5', key='all_votes', mode='w')

        print(this_filename)
        # time.sleep(2)
    except Exception as e:
        print(e)
        
    


0it [00:00, ?it/s]
1it [00:03,  3.89s/it]

'NoneType' object has no attribute 'tbody'



2it [00:06,  3.56s/it]

'NoneType' object has no attribute 'tbody'



3it [00:09,  3.33s/it]

...

9_1_50.csv



3654it [14:51:11, 12.16s/it]

9_1_51.csv



3655it [14:51:13,  9.15s/it]


'NoneType' object has no attribute 'tbody'


In [14]:
chk_votes = pd.read_hdf('all_votes_pending.h5')

In [15]:
chk_votes[ chk_votes['saved_as'] == '' ].shape

(139, 7)